In [ ]:
# -*- coding: utf-8 -*-
# Javier Gálvez Obispo

from google.colab import drive
drive.mount('/content/gdrive')
path = "gdrive/My Drive/imagenes/" # path usando Colab
# path = "./imagenes"              # path usando Spyder

In [ ]:
###########################################
####### CARGAR LIBRERÍAS NECESARIAS #######
###########################################

# En caso de necesitar instalar keras en google colab,
# ejecutar la siguiente línea:
# !pip install -q keras
# Importar librerías necesarias
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import keras.utils as np_utils

# Importar modelos y capas que se van a usar
from keras import Sequential, Model
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
from keras.layers import BatchNormalization, Dropout, Add, Input

# Preprocesado
from keras.preprocessing.image import load_img,img_to_array
from keras.preprocessing.image import ImageDataGenerator

# Callbacks
from keras.callbacks import EarlyStopping

# Importar el optimizador a usar
from keras.optimizers import SGD, Adam

# Importar el conjunto de datos
from keras.datasets import cifar100

# ResNet
from keras.applications.resnet50 import ResNet50, preprocess_input

# Eliminar Warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#######################################
####### FUNCIONES IMPLEMENTADAS #######
#######################################

####### FUNCION DE LECTURA PARA CIFAR100 #######

# A esta función solo se la llama una vez. Devuelve 4 
# vectores conteniendo, por este orden, las imágenes
# de entrenamiento, las clases de las imágenes de
# entrenamiento, las imágenes del conjunto de test y
# las clases del conjunto de test.
def cargarImagenesCifar():
    # Cargamos Cifar100. Cada imagen tiene tamaño
    # (32 , 32, 3). Nos vamos a quedar con las
    # imágenes de 25 de las clases.
    (x_train, y_train), (x_test, y_test) = cifar100.load_data (label_mode ='fine')
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    train_idx = np.isin(y_train, np.arange(25))
    train_idx = np.reshape (train_idx, -1)
    x_train = x_train[train_idx]
    y_train = y_train[train_idx]
    test_idx = np.isin(y_test, np.arange(25))
    test_idx = np.reshape(test_idx, -1)
    x_test = x_test[test_idx]
    y_test = y_test[test_idx]
    
    # Transformamos los vectores de clases en matrices.
    # Cada componente se convierte en un vector de ceros
    # con un uno en la componente correspondiente a la
    # clase a la que pertenece la imagen. Este paso es
    # necesario para la clasificación multiclase en keras.
    y_train = np_utils.to_categorical(y_train, 25)
    y_test = np_utils.to_categorical(y_test, 25)
    
    return x_train , y_train , x_test , y_test

####### FUNCIONES DE LECTURA PARA CALTECH-UCSD #######

# Descargar las imágenes de http://www.vision.caltech.edu/visipedia/CUB-200.html
# Descomprimir el fichero.
# Descargar también el fichero list.tar.gz, descomprimirlo y guardar los ficheros
# test.txt y train.txt dentro de la carpeta de imágenes anterior. Estos 
# dos ficheros contienen la partición en train y test del conjunto de datos.
# EN CASO DE USAR COLABORATORY
# Sube tanto las imágenes como los ficheros text.txt y train.txt a tu drive.

# Dado un fichero train.txt o test.txt y el path donde se encuentran los
# ficheros y las imágenes, esta función lee las imágenes
# especificadas en ese fichero y devuelve las imágenes en un vector y 
# sus clases en otro.
def leerImagenes(vec_imagenes, path):
    clases = np.array([img.split('/')[0] for img in vec_imagenes])
    imagenes = np.array([img_to_array(load_img(path + "/" + img,
                                               target_size = (224, 224))) 
                        for img in vec_imagenes])
    return imagenes, clases


# Usando la función anterior, y dado el path donde se encuentran las
# imágenes y los archivos "train.txt" y "test.txt", devuelve las 
# imágenes y las clases de train y test para usarlas con keras
# directamente.
def cargarDatosCaltech(path):
    # Cargamos los ficheros
    train_images = np.loadtxt(path + "/train.txt", dtype = str)
    test_images = np.loadtxt(path + "/test.txt", dtype = str)

    # Leemos las imágenes con la función anterior
    train, train_clases = leerImagenes(train_images, path)
    test, test_clases = leerImagenes(test_images, path)

    # Pasamos los vectores de las clases a matrices 
    # Para ello, primero pasamos las clases a números enteros
    clases_posibles = np.unique(np.copy(train_clases))
    for i in range(len(clases_posibles)):
        train_clases[train_clases == clases_posibles[i]] = i
        test_clases[test_clases == clases_posibles[i]] = i

    # Después, usamos la función to_categorical()
    train_clases = np_utils.to_categorical(train_clases, 200)
    test_clases = np_utils.to_categorical(test_clases, 200)

    # Barajar los datos
    train_perm = np.random.permutation(len(train))
    train = train[train_perm]
    train_clases = train_clases[train_perm]

    test_perm = np.random.permutation(len(test))
    test = test[test_perm]
    test_clases = test_clases[test_perm]

    return train, train_clases, test, test_clases

# Función auxiliar para leer los datos desde un archivo en vez de leer cada imagen una por una
# Ha sido necesario cambiar el método de lectura ya que en Colab tardaba demasiado en leer las imágenes
def cargarDatosCaltech_aux(path):
    # Leer imágenes desde binario
    x_train = np.load(path + 'x_train.npy')
    y_train = np.load(path + 'y_train.npy')
    x_test = np.load(path + 'x_test.npy')
    y_test = np.load(path + 'y_test.npy')

    # Barajado de las imágenes
    train_perm = np.random.permutation(len(x_train))
    test_perm = np.random.permutation(len(x_test))
    x_train = x_train[train_perm]
    y_train = y_train[train_perm]
    x_test = x_test[test_perm]
    y_test = y_test[test_perm]

    return x_train, y_train, x_test, y_test

###### FUNCINOES GENERALES UTLIZADAS EN TODOS LOS EJERCICIOS #######

# Función para entrenar un modelo dado junto con los generadores de los conjuntos
def entrenar(model, train, validation, epocas, early_stopping):
  
    callbacks = []

    # Early Stopping es una de las mejoras que se pueden aplicar en el apartado 2
    if early_stopping:
        perdida = EarlyStopping(monitor="val_loss",
                                patience=patience, 
                                restore_best_weights=True)
        precision = EarlyStopping(monitor="val_accuracy",
                                  patience=patience,
                                  restore_best_weights=True)
        callbacks.append(perdida)
        callbacks.append(precision)

    # Entrenamos
    hist = model.fit_generator(train, epochs=epocas, verbose=1,
                               validation_data=validation,
                               callbacks=callbacks)
    
    return hist

# Función para evaluar un modelo dado en el conjunto de test y mostrar los resultados
def evaluar(model, x_test, y_test, datagen=None):
    print("----------- RESULTADOS -----------")
    if datagen == None:
        score = model.evaluate(x_test, y_test, verbose=0)
    else:
        score = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=1, shuffle=False),
                                        verbose=0, steps=len(x_test))
  
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print("----------------------------------")
    input("## Pulse tecla para continuar ##\n")

    return (score[0], score[1])

# Esta función pinta dos gráficas, una con la evolución
# de la función de pérdida en el conjunto de train y
# en el de validación, y otra con la evolución de la
# accuracy en el conjunto de train y el de validación.
# Es necesario pasarle como parámetro el historial del
# entrenamiento del modelo (lo que devuelven las
# funciones fit() y fit_generator()).
def mostrarEvolucion(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()

    input("## Pulse tecla para continuar ##\n")
    
    acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']
    plt.plot(acc)
    plt.plot(val_acc)
    plt.legend(['Training accuracy','Validation accuracy'])
    plt.show()

    input("## Pulse tecla para continuar ##\n")

# Función para comprar resultados de 2 modelos
def compararHist(hist1, hist2, label1, label2, resultado):
    print("Linea fina   = train\nLinea gruesa = validation\n")
    res1 = hist1.history[resultado]
    val_res1 = hist1.history['val_' + resultado]
    res2 = hist2.history[resultado]
    val_res2 = hist2.history['val_' + resultado]
    plt.plot(res1, color='b', lw=1.0)
    plt.plot(val_res1, color='b', lw=3.0, label=label1)
    plt.plot(res2, color='r', lw=1.0)
    plt.plot(val_res2, color='r', lw=3.0, label=label2)
    plt.legend()
    plt.show()
    input("## Pulse tecla para continuar ##\n")

####### FUNCIONES DE BASENET #######

### BaseNet ###

# Función que devuelve el modelo de BaseNet básico
# El parámetro batch_normalization se utiliza en el apartado 2 para poner, o no, capas de BatchNormalization
def basenet(batch_normalization=False):
    model = Sequential()

    # Convolución 5x5x6
    model.add(Conv2D(filters=6, kernel_size=5, input_shape=input_shape))
    if batch_normalization: model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolución 5x5x16
    model.add(Conv2D(filters=16, kernel_size=5))
    if batch_normalization: model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(50))
    if batch_normalization:
      model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    model.add(Dense(25, activation="softmax"))
    
    return model

### BaseNet con las mejoras del ejercicio 2 ###

# Función que devuelve el modelo de BaseNet mejorado
# El parámetro dropout se utiliza para poner, o no, capas de Dropout
def basenet_mejorado(dropout=True):
    model = Sequential()

    # Convolución 5x5x6
    model.add(Conv2D(filters=6, kernel_size=5, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    # Convolución 3x3x24
    model.add(Conv2D(filters=24, kernel_size=3, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolución 5x5x16
    model.add(Conv2D(filters=16, kernel_size=5))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    # Convolución 3x3x64
    model.add(Conv2D(filters=64, kernel_size=3, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    model.add(MaxPooling2D(pool_size=(2,2)))

    # Reducimos el número de canales para seguir teniendo un vector de longitud 400
    model.add(Conv2D(filters=16, kernel_size=1, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    if dropout: model.add(Dropout(0.2))
    model.add(Flatten())

    model.add(Dense(50))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    if dropout: model.add(Dropout(0.2))
    model.add(Dense(25, activation="softmax"))
    
    return model

### BaseNet con las mejoras del bonus ###

# Función para añadir una capa de BatchNormalization y otra de ReLU a un modelo
def bn_relu(input):
    bn = BatchNormalization()(input)
    relu = Activation(keras.activations.relu)(bn)
    return relu

# Función para añadir un bloque convolucional plano
def bloque_plano(input, filters, kernel_size=3):
    # Primera convolución
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(input)
    x = bn_relu(x)

    # Segunda convolución
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(x)
    x = bn_relu(x)

    return x

# Función para añadir un bloque convolucional residual
def bloque_residual(input, filters, kernel_size=3):
    # Primera convolución
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(input)
    x = bn_relu(x)

    # Segunda convolución
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(x)
    
    # Sumamos la entrada del bloque con la salida
    output = Add()([input, x])
    output = bn_relu(output)

    return output

# Función que devuelve BaseNet con todas las mejoras del bonus
def basenet_bonus(residual=False):
    inputs = Input(shape=input_shape)
    filtros = 32

    # Primera convolución 5x5
    x = Conv2D(filters=filtros, kernel_size=5, padding="same")(inputs)
    x = bn_relu(x)

    # Bloque de convoluciones
    x =  bloque_residual(x, filters=filtros) if residual else bloque_plano(x, filters=filtros)

    x = MaxPooling2D((2, 2))(x)

    # Segunda convolución 5x5
    x = Conv2D(filters=filtros*2, kernel_size=5, padding="same")(x)
    x = bn_relu(x)
    
    # Bloque de convoluciones
    x =  bloque_residual(x, filters=filtros*2) if residual else bloque_plano(x, filters=filtros*2)

    x = MaxPooling2D((2, 2))(x)

    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = bn_relu(x)

    x = Dropout(0.5)(x) 
    output = Dense(25, activation="softmax")(x)

    model = Model(inputs, output)
    
    return model

### Test de BaseNet ###

# Función general para probar todos las variantes de BaseNet
def probar_basenet(preprocesado, variante, sgd=False, epocas=25, summary=False, 
                   batch_normalization=False, dropout=False, early_stopping=False, 
                   residual=True):

    # Cuando normalizamos standardize modifica el array original, es necesario 
    # generar una copia si queremos llamar varias veces a la función
    test = np.copy(x_test)

    # Preprocesado de los datos + Data Augmentation
    datagen = ImageDataGenerator(**preprocesado)
    datagen.fit(x_train)

    # Creamos los generadores
    train = datagen.flow(x_train, y_train,
                        batch_size=batch_size,
                        subset="training")
    validation = datagen.flow(x_train, y_train,
                            batch_size=batch_size,
                            subset="validation")
    # Si hemos normalizado train hacemos las mismas operaciones en test
    if "featurewise_std_normalization" in preprocesado:
        datagen.standardize(test)

    # Creamos el modelo
    if variante == "BaseNet":
        modelo = basenet(batch_normalization)
    elif variante == "Mejorado":
        modelo = basenet_mejorado(dropout)
    elif variante == "Bonus":
        modelo = basenet_bonus(residual)

    # Imprimir por pantalla el modelo
    if summary:
        modelo.summary()
        input("## Pulse tecla para continuar ##\n")

    # Compilamos el modelo
    if sgd:
        opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    else:
        opt = Adam()

    modelo.compile(loss=keras.losses.categorical_crossentropy,
                   optimizer=opt, metrics=metrica)

    # Entrenamos y evaluamos el modelo
    hist = entrenar(modelo, train, validation, epocas, early_stopping)
    print("\nEVALUAMOS EL MODELO\n")
    input("## Pulse tecla para continuar ##\n")
    mostrarEvolucion(hist)
    score = evaluar(modelo, test, y_test)

    return hist, score

####### FUNCIONES DE RESNET #######

# Función que devuelve el modelo más sencillo posible, una capa de salida
def resnet_simple():
    model = Sequential()
    model.add(Input(shape=(2048,)))
    model.add(Dense(200, activation = 'softmax'))
    return model

# Función que devuelve un modelo con 2 capas fully-connected más capa de salida
def modelo_A():
    model = Sequential()

    model.add(Input(shape=(2048,)))

    # Primera capa FC
    model.add(Dropout(0.4))
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    # Segunda capa FC
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    # Salida
    model.add(Dense(200, activation = 'softmax'))

    return model

# Función que le añade una capa convolucional al mismo modelo devuelto por modela_A()
def modelo_B():
    model = Sequential()

    # Capa convolucional
    model.add(Conv2D(filters=64, kernel_size=3, input_shape=(7, 7, 2048), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    # Primera capa FC
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1024, input_shape=(2048,)))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    # Segunda capa FC
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation(keras.activations.relu))

    # Salida
    model.add(Dense(200, activation = 'softmax'))

    return model

# Función que extrae las característica de un conjunto de datos
def obtener_caracteristicas(model, datagen, x):
    caracteristicas = model.predict_generator(datagen.flow(x, batch_size=1, shuffle=False),
                                              verbose=1,
                                              steps=len(x))
    return caracteristicas

### Extracción de características con ResNet50 ###

# Función para utilizar ResNet50 como extractor de características
def resnet_extractor(apartado):
    
    # Creamos el modelo a utilizar y configuramos ResNet50
    if apartado == 'A':
        pooling_type = 'avg'
        modelo = modelo_A()
    elif apartado == 'B':
        pooling_type = None
        modelo = modelo_B()
    elif apartado == 'Base':
        pooling_type = 'avg'
        modelo = resnet_simple()

    # Cargamos ResNet50 entrenado con imagenet
    resnet = ResNet50(weights='imagenet', include_top=False, pooling=pooling_type)
    
    # Preprocesado de ResNet
    datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

    # Extraemos características de las imágenes con el modelo anterior
    print("Extrayendo caractarísticas de train")
    caracteristicas_train = obtener_caracteristicas(resnet, datagen, x_train)
    print("Extrayendo caractarísticas de test")
    caracteristicas_test = obtener_caracteristicas(resnet, datagen, x_test)

    modelo.summary()

    # Compilamos el modelo
    modelo.compile(loss=keras.losses.categorical_crossentropy,
                   optimizer=Adam(),
                   metrics=metrica)

    # Entrenamos y evaluamos el modelo
    hist = modelo.fit(caracteristicas_train, y_train, batch_size = batch_size,
                      epochs = epocas, verbose = 1, validation_split = 0.1)
    print("\nEVALUAMOS EL MODELO\n")
    input("## Pulse tecla para continuar ##\n")
    mostrarEvolucion(hist)
    score = evaluar(modelo, caracteristicas_test, y_test)

    return hist, score

### Fine-tuning de ResNet a Caltech-UCSD ###

# Función para realizar un ajuste fino de ResNet50 a un conjunto de datos
def resnet_fine_tuning(epocas=10):
    # Cargamos ResNet50 entrenado con imagenet
    resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

    # Creamos los generadores
    train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input, validation_split=0.1)
    test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

    train = train_datagen.flow(x_train, y_train,batch_size=batch_size, subset="training")
    validation = train_datagen.flow(x_train, y_train, batch_size=batch_size, subset="validation")

    # Añadimos la capa de salida nueva
    x = resnet.output
    last = Dense(200, activation='softmax')(x)
    modelo = Model(inputs=resnet.input, outputs=last)

    # Compilamos
    modelo.compile(loss=keras.losses.categorical_crossentropy,
                   optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
                   metrics=metrica)

    # Entrenamos y evaluamos el modelo
    hist = entrenar(modelo, train, validation, epocas=epocas, early_stopping=False)
    print("\nEVALUAMOS EL MODELO\n")
    input("## Pulse tecla para continuar ##\n")
    mostrarEvolucion(hist)
    score = evaluar(modelo, x_test, y_test, test_datagen)

    return hist, score

#########################################
####### PRUEBAS DE LOS EJERCICIOS #######
#########################################

# Variables generales
epocas = 25                             # Número de épocas realizadas al probar un modelo
patience = 10                           # Número de épocas sin mejoras antes de parar
batch_size = 64                         # Tamaño del batch a utilizar
input_shape = (32, 32 ,3)               # Dimensiones de los datos de entrada
metrica = ['accuracy']                  # Métrica que vamos a utilizar
preprocesado = dict()                   # Diccionario con el valor de todos los parámetros de preprocesado
preprocesado["validation_split"] = 0.1  # Porcentaje del conjunto de train que se usa para validación

print("\n\n------------ Ejercicio 1 ------------\n\n")

# Leemos los datos de Cifar100
print("Leyendo los datos de Cifar100")
x_train , y_train , x_test , y_test = cargarImagenesCifar()
print("Datos leidos\n")

print("Comparamos los resultados de BaseNet utilizando distintos optimizadores\n")

# Probar BaseNet utilizando el SGD dado
print("Probamos BaseNet con el SGD dado")
input("## Pulse tecla para continuar ##\n")
hist_sgd, score_sgd = probar_basenet(preprocesado, "BaseNet", sgd=True, epocas=epocas, summary=True)

# Probar BaseNet utilizando Adam
print("Probamos BaseNet con Adam")
input("## Pulse tecla para continuar ##\n")
hist_adam, score_adam = probar_basenet(preprocesado, "BaseNet", sgd=False, epocas=epocas)

print("Mostramos gráficas comparando ambos resultados")

print("Comparamos las pérdidas en train y validation para ambos")
input("## Pulse tecla para continuar ##\n")
compararHist(hist_sgd, hist_adam, "SGD", "Adam", "loss")

print("Comparamos la precisión en train y validation para ambos")
input("## Pulse tecla para continuar ##\n")
compararHist(hist_sgd, hist_adam, "SGD", "Adam", "accuracy")


print("\n\n------------ Ejercicio 2 ------------\n\n")

print("Añadimos mejoras al modelo básico de BaseNet")
### Sólo se ejecuta la versión definitiva con todas las mejoras ###

## Normalización
preprocesado["featurewise_center"] = True
preprocesado["featurewise_std_normalization"] = True

# print("Vemos los resultados añadiendo normalización")
# input("## Pulse tecla para continuar ##\n")
# probar_basenet(preprocesado, "BaseNet", epocas=epocas)

## Data Augmentation
preprocesado["horizontal_flip"] = True
preprocesado["zoom_range"] = 0.2
preprocesado["width_shift_range"] = 0.1
preprocesado["height_shift_range"] = 0.1
preprocesado["rotation_range"] = 20

# print("Vemos los resultados añadiendo data augmentation")
# input("## Pulse tecla para continuar ##\n")
# probar_basenet(preprocesado, "BaseNet", epocas=epocas)

## BatchNormalization
# print("Vemos los resultados añadiendo batch normalization")
# input("## Pulse tecla para continuar ##\n")
# probar_basenet(preprocesado, "BaseNet", epocas=epocas, batch_normalization=True)

## Añadimos Aumento de profundidad
# print("Vemos los resultados aumentando la profundidad")
# input("## Pulse tecla para continuar ##\n")
# probar_basenet(preprocesado, "Mejorado", epocas=epocas)

## Añadimos Dropout
# print("Vemos los resultados introduciendo dropout")
# input("## Pulse tecla para continuar ##\n")
# probar_basenet(preprocesado, "Mejorado", epocas=epocas, dropout=True)

## Early Stopping 
## VERSIÓN DEFINITIVA ##
# print("Vemos los resultados introduciendo early stopping")
print("Versión definitiva de BaseNet con todas las mejoras")
input("## Pulse tecla para continuar ##\n")
probar_basenet(preprocesado, "Mejorado", epocas=80, dropout=True, early_stopping=True, summary=True)


print("\n\n------------ Ejercicio 3 ------------\n\n")

print("Leyendo los datos de Caltech-UCSD")
# Leer las imágenes una a una de las carpetas
x_train, y_train, x_test, y_test = cargarDatosCaltech(path)

# Leer directamente los numpy array ya guardados
# x_train, y_train, x_test, y_test = cargarDatosCaltech_aux(path)
print("Datos leidos\n")

epocas = 25

print("ResNet como extractor de características\n")
input("## Pulse tecla para continuar ##\n")

print("Sustituimos la capa de salida de ResNet")
input("## Pulse tecla para continuar ##\n")
resnet_extractor('Base')

print("Añadimos capas fully-connected antes de la capa de salida")
input("## Pulse tecla para continuar ##\n")
resnet_extractor('A')

print("Añadimos una capa de convolución antes de las capas fully-connected")
input("## Pulse tecla para continuar ##\n")
resnet_extractor('B')


print("Ajuste fino de ResNet para el conjunto de Caltech-UCSD\n")
input("## Pulse tecla para continuar ##\n")
resnet_fine_tuning()


print("\n\n------------ Bonus ------------\n\n")

# Leemos los datos de Cifar100
print("Leyendo los datos de Cifar100")
x_train , y_train , x_test , y_test = cargarImagenesCifar()
print("Datos leidos\n")

print("BaseNet Bonus")
input("## Pulse tecla para continuar ##\n")

epocas = 80
# Versión plana
# probar_basenet(preprocesado, "Bonus", epocas=epocas, early_stopping=True, residual=False)

# Versión residual
probar_basenet(preprocesado, "Bonus", epocas=epocas, early_stopping=True, residual=True, summary=True)
